<a href="https://colab.research.google.com/github/djbell01/563-DawsonBell/blob/main/Week_3_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte Carlo

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Grid
class grid():
  def __init__(self,length,N):
    # Create a square grid with N particles
    self.grid_spaces = length**2
    self.N = N
    self.rng = np.random.default_rng()

    # Generate random positions for N particles
    self.particles = []
    for i in range(N):
      x = self.rng.integers(0,length,1)
      y = self.rng.integers(0,length,1)
      self.particles.append(particle(x,y))

    # Check for overlapping particles, move if necessary

  def move_particle(self,index):
    # Determine where all empty spaces are and choose one at random.
    self.empty_spaces = []
    for i in range(self.length):
      for j in range(self.length):
        for particle in self.particles:
          if particle.x == i and particle.y == j:
            continue
          else:
            self.empty_spaces.append([i,j])

    rand = self.rng.integers(0,len(self.empty_spaces),1)
    rand_empty_space = self.empty_spaces[rand]
    self.particles[index].x = rand_empty_space[0]
    self.particles[index].y = rand_empty_space[1]


    #self.particles[index]

# Particle
class particle():
  def __init__(self,x,y):
    self.x = x
    self.y = y




In [10]:
rng = np.random.default_rng()
rand = rng.integers(0,10,1)
print(rand)

[2]
